In [50]:
from django.db import models
from django.db.models.sql.query import Query
from django.db.models.sql.datastructures import Join
from django.db.models.query import ModelIterable
from django.contrib.contenttypes.models import ContentType
from django.contrib.contenttypes.fields import GenericRelation


class IncludeModelIterable(ModelIterable):
    
    def __iter__(self):
        print(self.queryset._includes)
        for instance in super(IncludeModelIterable, self).__iter__():
            for includer in self.queryset._includes.values():
                includer.extract(instance)
                
            yield instance
            
            
class AbstractIncluder(object):
    def __init__(self, alias, field):
        self.alias = alias
        self.field = field
        self.model = field.related_model
        self.host_model = field.model
        

class ManyToOneIncluder(AbstractIncluder):
    
    def extract(self, instance):
        data = getattr(instance, self.alias)
        delattr(instance, self.alias)
        setattr(instance, self.field.get_cache_name(), self.model.from_db(instance._state.db, None, data))
        
    def extra_query(self):
        return {self.alias:'''
        SELECT json_build_array({select})
        FROM "{table}" AS "{alias}"
        WHERE "{alias}"."{join_column}" = "{host}"."{host_join_column}"
        '''.format(
            alias=self.alias,
            table=self.model._meta.db_table,
            join_column=self.field.target_field.column,
            host=self.host_model._meta.db_table,
            host_join_column=self.field.column,
            select=', '.join('"{}"."{}"'.format(self.alias, f.column) for f in self.model._meta.concrete_fields)
        )}


class OneToManyIncluder(AbstractIncluder):
    
    def extract(self, instance):
        data = getattr(instance, self.alias)
        delattr(instance, self.alias)
        
        if not hasattr(instance, '_prefetched_objects_cache'):
            instance._prefetched_objects_cache = {}
        instance._prefetched_objects_cache[self.field.name] = self.model.objects.none()
        instance._prefetched_objects_cache[self.field.name]._result_cache = [
            self.model.from_db(instance._state.db, None, datum)
            for datum in data
        ]
    
    def extra_query(self):
        host_join_column, join_column = self.field.get_joining_columns()[0]
        return {self.alias:'''
        SELECT json_agg(json_build_array({select}))
        FROM "{table}" AS "{alias}"
        WHERE "{alias}"."{join_column}" = "{host}"."{host_join_column}"
        '''.format(
            alias=self.alias,
            table=self.model._meta.db_table,
            join_column=join_column,
            host=self.host_model._meta.db_table,
            host_join_column=host_join_column,
            select=', '.join('"{}"."{}"'.format(self.alias, f.column) for f in self.model._meta.concrete_fields)
        )}
    
    
class OneToManyIncluder(AbstractIncluder):
    
    def extract(self, instance):
        data = getattr(instance, self.alias)
        delattr(instance, self.alias)
        
        if not hasattr(instance, '_prefetched_objects_cache'):
            instance._prefetched_objects_cache = {}
        instance._prefetched_objects_cache[self.field.name] = self.model.objects.none()
        instance._prefetched_objects_cache[self.field.name]._result_cache = [
            self.model.from_db(instance._state.db, None, datum)
            for datum in data
        ]
    
    def extra_query(self):
        host_join_column, join_column = self.field.get_joining_columns()[0]
        return {self.alias:'''
        SELECT json_agg(json_build_array({select}))
        FROM "{table}" AS "{alias}"
        WHERE "{alias}"."{join_column}" = "{host}"."{host_join_column}"
        '''.format(
            alias=self.alias,
            table=self.model._meta.db_table,
            join_column=join_column,
            host=self.host_model._meta.db_table,
            host_join_column=host_join_column,
            select=', '.join('"{}"."{}"'.format(self.alias, f.column) for f in self.model._meta.concrete_fields)
        )}

    
class GenericRelationIncluder(OneToManyIncluder):
    def extra_query(self):
        join_column, host_join_column = self.field.get_joining_columns()[0]
        return {self.alias:'''
        SELECT json_agg(json_build_array({select}))
        FROM "{table}" AS "{alias}"
        WHERE "{alias}"."{join_column}" = "{host}"."{host_join_column}"
        AND "{alias}"."{content_type}" = {content_type_id}
        '''.format(
            alias=self.alias,
            table=self.model._meta.db_table,
            join_column=join_column,
            host=self.host_model._meta.db_table,
            host_join_column=host_join_column,
            select=', '.join('"{}"."{}"'.format(self.alias, f.column) for f in self.model._meta.concrete_fields),
            content_type=self.model._meta.get_field(self.field.content_type_field_name).column,
            content_type_id=ContentType.objects.get_for_model(self.host_model).pk
        )}


class IncludeQuerySet(models.QuerySet):

    def __init__(self, *args, **kwargs):
        super(IncludeQuerySet, self).__init__(*args, **kwargs)
        self._includes = {}
        self._iterable_class = IncludeModelIterable

    def include(self, *related_names):
        clone = self._clone()
        for related_name in related_names:
            field = self.model._meta.get_field(related_name)
            if isinstance(field, GenericRelation):
                clone._includes[field] = GenericRelationIncluder('__garbage__' + field.name, field)
            elif field.many_to_one:
                clone._includes[field] = ManyToOneIncluder('__garbage__' + field.name, field)
            elif field.one_to_many:
                clone._includes[field] = OneToManyIncluder('__garbage__' + field.name, field)
        clone.__construct_extra()
        return clone
    
    def __construct_extra(self):
        for includer in self._includes.values():
            self.query.add_extra(includer.extra_query(), None, None, None, None, None)
            
    def _clone(self):
        clone = super(IncludeQuerySet, self)._clone()
        clone._includes = self._includes
        return clone
        
IncludeManager = IncludeQuerySet.as_manager()
IncludeManager.contribute_to_class(AbstractNode, 'bobjects')

In [188]:
from django.db import models
from django.db.models.sql.query import Query
from django.db.models.sql.datastructures import Join
from django.db.models.query import ModelIterable
from django.contrib.contenttypes.models import ContentType
from django.contrib.contenttypes.fields import GenericRelation
from django.db.models.expressions import Func
from django.contrib.postgres.aggregates import ArrayAgg
from django.contrib.postgres.fields import JSONField


class JSONBuildArray(Func):
    function = 'JSON_BUILD_ARRAY'

    
class JSONAgg(ArrayAgg):
    function = 'JSON_AGG'
    def __init__(self, *args):
        super(JSONAgg, self).__init__(*args, output_field=JSONField())



class IncludeModelIterable(ModelIterable):
    
    def __iter__(self):
        print(self.queryset._includes)
        for instance in super(IncludeModelIterable, self).__iter__():
            for includer in self.queryset._includes.values():
                includer.extract(instance)
                
            yield instance
            
            
class IncludeQuerySet(models.QuerySet):

    def __init__(self, *args, **kwargs):
        super(IncludeQuerySet, self).__init__(*args, **kwargs)
#         self._includes = {}
#         self._iterable_class = IncludeModelIterable

    def include(self, *related_names):
        clone = self._clone()
        for related_name in related_names:
            clone._include(self.model._meta.get_field(related_name))
        return clone

    def _include(self, field):
        model = field.related_model
        column, host_column = field.get_joining_columns()[0]
        qs = model.objects.all().extra(where=['"{table}"."{column}" = "{host_table}"."{host_column}"'.format(
            table=model._meta.db_table,
            column=column,
            host_table=model._meta.db_table,
            host_column=host_column,
        )])
        qs.query.add_annotation(
            JSONAgg(JSONBuildArray(*[f.column for f in model._meta.concrete_fields])),
            '__fields',
            is_summary=True
        )
        qs = qs.values_list('__fields')
        qs.query.order_by = []
        qs.query.default_ordering = False
        sql, params = qs.query.sql_with_params()
        
        self.query.add_extra({'__' + field.name: sql}, params, None, None, None, None)
            
#     def _clone(self):
#         clone = super(IncludeQuerySet, self)._clone()
#         clone._includes = self._includes
#         return clone
        
IncludeManager = IncludeQuerySet.as_manager()
IncludeManager.contribute_to_class(AbstractNode, 'bobjects')

In [189]:
print(str(AbstractNode.bobjects.include('guids').query))

SELECT (SELECT JSON_AGG(JSON_BUILD_ARRAY("osf_guid"."id", "osf_guid"."_id", "osf_guid"."content_type_id", "osf_guid"."object_id", "osf_guid"."created")) AS "__fields" FROM "osf_guid" WHERE ("osf_guid"."object_id" = "osf_guid"."id")) AS "__guids", "osf_abstractnode"."id", "osf_abstractnode"."guid_string", "osf_abstractnode"."content_type_pk", "osf_abstractnode"."type", "osf_abstractnode"."spam_status", "osf_abstractnode"."spam_pro_tip", "osf_abstractnode"."spam_data", "osf_abstractnode"."date_last_reported", "osf_abstractnode"."reports", "osf_abstractnode"."category", "osf_abstractnode"."child_node_subscriptions", "osf_abstractnode"."creator_id", "osf_abstractnode"."date_created", "osf_abstractnode"."date_modified", "osf_abstractnode"."deleted_date", "osf_abstractnode"."description", "osf_abstractnode"."file_guid_to_share_uuids", "osf_abstractnode"."forked_date", "osf_abstractnode"."forked_from_id", "osf_abstractnode"."is_fork", "osf_abstractnode"."is_public", "osf_abstractnode"."is_del

In [190]:
AbstractNode.bobjects.include('guids')[0].__dict__

{'__guids': [[1, u'zv7n5', 18, 1, u'2017-02-17T16:45:57.574252+00:00'],
  [2, u'zv7jc', 18, 2, u'2017-02-17T16:45:57.574772+00:00'],
  [3, u'ztuyr', 18, 3, u'2017-02-17T16:45:57.574796+00:00'],
  [4, u'ztuyc', 18, 4, u'2017-02-17T16:45:57.574805+00:00'],
  [5, u'zth62', 18, 5, u'2017-02-17T16:45:57.574816+00:00'],
  [6, u'zs6uv', 18, 6, u'2017-02-17T16:45:57.574825+00:00'],
  [7, u'zr68w', 18, 7, u'2017-02-17T16:45:57.574833+00:00'],
  [8, u'zqrk6', 18, 8, u'2017-02-17T16:45:57.57484+00:00'],
  [9, u'znqjk', 18, 9, u'2017-02-17T16:45:57.574848+00:00'],
  [10, u'znhcq', 18, 10, u'2017-02-17T16:45:57.574856+00:00'],
  [11, u'zjndq', 18, 11, u'2017-02-17T16:45:57.574864+00:00'],
  [12, u'zh82t', 18, 12, u'2017-02-17T16:45:57.574871+00:00'],
  [1068, u'2tq6r', 18, 1068, u'2017-02-17T16:45:57.588223+00:00'],
  [1069, u'2tf4c', 18, 1069, u'2017-02-17T16:45:57.588235+00:00'],
  [1070, u'2szr4', 18, 1070, u'2017-02-17T16:45:57.588246+00:00'],
  [1071, u'2rmuw', 18, 1071, u'2017-02-17T16:45:57.

In [159]:
field = AbstractNode._meta.get_field('guids')

In [160]:
field.get_joining_columns()

(('object_id', u'id'),)

In [154]:
from django.db.models.expressions import Func
from django.contrib.postgres.aggregates import ArrayAgg
from django.contrib.postgres.fields import JSONField


class JSONBuildArray(Func):
    function = 'json_build_array'

    
class JSONAgg(ArrayAgg):
    function = 'JSON_AGG'
    def __init__(self, *args):
        super(JSONAgg, self).__init__(*args, output_field=JSONField())


qs = Guid.objects.all().extra(where=['osf_guid.object_id = osf_abstractnode.id'])
qs.query.add_annotation(JSONAgg(JSONBuildArray('id', '_id')), 'ids', is_summary=True)
qs = qs.values_list('ids')
qs.query.order_by = []
qs.query.default_ordering = False
print(qs.query.__dict__)
sql, params = qs.query.sql_with_params()
print(sql)
AbstractNode.objects.all().extra(select={'_guids': sql}, select_params=params)

{'context': {}, 'extra_order_by': (), 'extra_tables': (), '_extra': None, 'select': [], 'tables': [u'osf_guid'], 'used_aliases': set([]), 'order_by': [], 'distinct': False, '_annotation_select_cache': None, 'group_by': None, 'select_for_update': False, 'select_related': False, 'deferred_loading': (set([]), True), 'table_map': {u'osf_guid': [u'osf_guid']}, 'alias_map': {u'osf_guid': <django.db.models.sql.datastructures.BaseTable object at 0x11650f9d0>}, 'max_depth': 5, 'standard_ordering': True, 'distinct_fields': [], 'annotation_select_mask': set(['ids']), 'extra_select_mask': set([]), 'select_for_update_nowait': False, '_extra_select_cache': None, 'low_mark': 0, 'external_aliases': set([]), 'alias_refcount': {u'osf_guid': 3}, 'values_select': [], 'default_cols': False, '_annotations': OrderedDict([('ids', JSONAgg(JSONBuildArray(Col(osf_guid, osf.Guid.id), Col(osf_guid, osf.Guid._id))))]), 'default_ordering': False, 'where_class': <class 'django.db.models.sql.where.WhereNode'>, 'model'

[<Registration: Proj : (e6248)>, <Registration: a registration project : (e5w73)>, <Registration: Comp2 : (e5pkg)>, <Registration: Clouds : (e5gcm)>, <Registration: Comp3 : (e59xc)>, <Registration: Test! : (e57fv)>, <Registration: Todd's test - Cars : (e53gc)>, <Registration: cheep : (e4ynj)>, <Registration: Green : (e4yjf)>, <Registration: Comp D4 : (e4vtn)>, <Node: Bulk stress 2016-06-10T12:19:25.983Z - 02 : (fy2vc)>, <Node: Bulk stress 2016-04-13T08:06:58.944Z - 03 : (fy2sb)>, <Node: Bulk stress 2016-07-12T00:41:47.288Z - 05 : (fy2hd)>, <Node: Bulk stress 2016-10-04T16:04:20.161Z - 03 : (fy26m)>, <Node: Bulk stress 2016-01-03T19:34:01.728Z - 10 : (fy24h)>, <Node: Bulk stress 2016-04-07T23:05:17.834Z - 01 : (fy24c)>, <Node: Bulk stress 2016-08-05T18:46:37.105Z - 02 : (fy23h)>, <Node: Bulk stress 2016-02-07T05:44:49.614Z - 01 : (fxzp8)>, <Node: Bulk stress 2016-09-24T03:01:06.532Z - 06 : (fxyrd)>, <Node: Bulk stress 2016-11-05T10:21:36.871Z - 08 : (fxy3p)>, '...(remaining elements tru

In [155]:
_[0].__dict__

{'_guids': [[67745, u'e6248']],
 '_has_abandoned_preprint': False,
 '_is_preprint_orphan': False,
 '_is_templated_clone': False,
 '_original_state': {},
 '_parent': None,
 '_state': <django.db.models.base.ModelState at 0x116441490>,
 'category': u'project',
 'child_node_subscriptions': {},
 'content_type_pk': 27,
 'creator_id': 97,
 'date_created': datetime.datetime(2015, 7, 1, 15, 40, 23, 20000, tzinfo=<UTC>),
 'date_last_reported': None,
 'date_modified': datetime.datetime(2015, 7, 1, 15, 40, 23, 20000, tzinfo=<UTC>),
 'deleted_date': None,
 'description': u'',
 'embargo_id': None,
 'embargo_termination_approval_id': None,
 'file_guid_to_share_uuids': {},
 'forked_date': None,
 'forked_from_id': None,
 'guid_string': [u'e6248'],
 'id': 156990,
 'is_bookmark_collection': None,
 'is_deleted': False,
 'is_fork': False,
 'is_public': True,
 'keenio_read_key': u'befcd1e4b9cc7a10ac26d1668a50a7c865e66f933fc60597eca940502881766f0d0c9cfc05b2e62465a7bfa7e538272bcab3c01503e2051d118de490008ad451

In [128]:
print(str(AbstractNode.objects.select_related('root').query))

SELECT "osf_abstractnode"."id", "osf_abstractnode"."guid_string", "osf_abstractnode"."content_type_pk", "osf_abstractnode"."type", "osf_abstractnode"."spam_status", "osf_abstractnode"."spam_pro_tip", "osf_abstractnode"."spam_data", "osf_abstractnode"."date_last_reported", "osf_abstractnode"."reports", "osf_abstractnode"."category", "osf_abstractnode"."child_node_subscriptions", "osf_abstractnode"."creator_id", "osf_abstractnode"."date_created", "osf_abstractnode"."date_modified", "osf_abstractnode"."deleted_date", "osf_abstractnode"."description", "osf_abstractnode"."file_guid_to_share_uuids", "osf_abstractnode"."forked_date", "osf_abstractnode"."forked_from_id", "osf_abstractnode"."is_fork", "osf_abstractnode"."is_public", "osf_abstractnode"."is_deleted", "osf_abstractnode"."node_license_id", "osf_abstractnode"."root_id", "osf_abstractnode"."piwik_site_id", "osf_abstractnode"."public_comments", "osf_abstractnode"."suspended", "osf_abstractnode"."template_node_id", "osf_abstractnode"."

In [130]:
AbstractNode.objects.filter(root__id=1).query.__dict__

{'_annotation_select_cache': None,
 '_annotations': None,
 '_extra': None,
 '_extra_select_cache': None,
 '_lookup_joins': [u'osf_abstractnode', 'T2'],
 'alias_map': {'T2': <django.db.models.sql.datastructures.Join at 0x116455590>,
  u'osf_abstractnode': <django.db.models.sql.datastructures.BaseTable at 0x116455610>},
 'alias_refcount': {'T2': 0, u'osf_abstractnode': 1},
 'annotation_select_mask': None,
 'context': {},
 'default_cols': True,
 'default_ordering': True,
 'deferred_loading': (set(), True),
 'distinct': False,
 'distinct_fields': [],
 'external_aliases': set(),
 'extra_order_by': (),
 'extra_select_mask': None,
 'extra_tables': (),
 'filter_is_sticky': False,
 'group_by': None,
 'high_mark': None,
 'low_mark': 0,
 'max_depth': 5,
 'model': osf.models.node.AbstractNode,
 'order_by': [],
 'select': [],
 'select_for_update': False,
 'select_for_update_nowait': False,
 'select_related': False,
 'standard_ordering': True,
 'table_map': {u'osf_abstractnode': [u'osf_abstractnode'

In [133]:
print(str(AbstractNode.objects.extra(tables=['osf_guid']).query))

SELECT "osf_abstractnode"."id", "osf_abstractnode"."guid_string", "osf_abstractnode"."content_type_pk", "osf_abstractnode"."type", "osf_abstractnode"."spam_status", "osf_abstractnode"."spam_pro_tip", "osf_abstractnode"."spam_data", "osf_abstractnode"."date_last_reported", "osf_abstractnode"."reports", "osf_abstractnode"."category", "osf_abstractnode"."child_node_subscriptions", "osf_abstractnode"."creator_id", "osf_abstractnode"."date_created", "osf_abstractnode"."date_modified", "osf_abstractnode"."deleted_date", "osf_abstractnode"."description", "osf_abstractnode"."file_guid_to_share_uuids", "osf_abstractnode"."forked_date", "osf_abstractnode"."forked_from_id", "osf_abstractnode"."is_fork", "osf_abstractnode"."is_public", "osf_abstractnode"."is_deleted", "osf_abstractnode"."node_license_id", "osf_abstractnode"."root_id", "osf_abstractnode"."piwik_site_id", "osf_abstractnode"."public_comments", "osf_abstractnode"."suspended", "osf_abstractnode"."template_node_id", "osf_abstractnode"."